In [7]:
import numpy as np

In [1]:
meta_file = "/home/duccd/Desktop/scannet_download/scans/scene0568_00/scene0568_00.txt"

In [2]:
lines = open(meta_file).readlines()

In [3]:
lines

['axisAlignment = -0.999657 0.026177 0.000000 2.822040 -0.026177 -0.999657 0.000000 3.740260 0.000000 0.000000 1.000000 -0.807330 0.000000 0.000000 0.000000 1.000000 \n',
 'colorHeight = 968\n',
 'colorToDepthExtrinsics = 0.999973 0.006791 0.002776 -0.037886 -0.006767 0.999942 -0.008366 -0.003410 -0.002833 0.008347 0.999961 -0.021924 -0.000000 0.000000 -0.000000 1.000000\n',
 'colorWidth = 1296\n',
 'depthHeight = 480\n',
 'depthWidth = 640\n',
 'fx_color = 1170.187988\n',
 'fx_depth = 571.623718\n',
 'fy_color = 1170.187988\n',
 'fy_depth = 571.623718\n',
 'mx_color = 647.750000\n',
 'mx_depth = 319.500000\n',
 'my_color = 483.750000\n',
 'my_depth = 239.500000\n',
 'numColorFrames = 1651\n',
 'numDepthFrames = 1651\n',
 'numIMUmeasurements = 3486\n',
 'sceneType = Living room / Lounge\n']

In [8]:
for line in lines:
    if 'axisAlignment' in line:
        axis_align_matrix = [float(x) for x in line.rstrip().strip('axisAlignment = ').split(' ')]

In [11]:
np.array(axis_align_matrix).reshape((4, 4))

array([[-0.999657,  0.026177,  0.      ,  2.82204 ],
       [-0.026177, -0.999657,  0.      ,  3.74026 ],
       [ 0.      ,  0.      ,  1.      , -0.80733 ],
       [ 0.      ,  0.      ,  0.      ,  1.      ]])

In [12]:
def read_aggregation(filename):
    object_id_to_segs = {}
    label_to_segs = {}
    with open(filename) as f:
        data = json.load(f)
        num_objects = len(data['segGroups'])
        for i in range(num_objects):
            object_id = data['segGroups'][i]['objectId'] + 1 # instance ids should be 1-indexed
            label = data['segGroups'][i]['label']
            segs = data['segGroups'][i]['segments']
            object_id_to_segs[object_id] = segs
            if label in label_to_segs:
                label_to_segs[label].extend(segs)
            else:
                label_to_segs[label] = segs
    return object_id_to_segs, label_to_segs

In [13]:
file = '/home/duccd/Desktop/InstanceRefer/data/scannet/scans/scene0011_00/scene0011_00.aggregation.json'

In [15]:
import json

In [17]:
a, b = read_aggregation(file)

In [20]:
len(b)

16

In [21]:
len(a)

33

In [4]:
file = '/home/duccd/Desktop/InstanceRefer/data/scannet/scans/scene0011_00/scene0011_00_vh_clean.segs.json'

In [8]:
import json

In [9]:
def read_segmentation(filename):
    seg_to_verts = {}
    with open(filename) as f:
        data = json.load(f)
        num_verts = len(data['segIndices'])
        for i in range(num_verts):
            seg_id = data['segIndices'][i]
            if seg_id in seg_to_verts:
                seg_to_verts[seg_id].append(i)
            else:
                seg_to_verts[seg_id] = [i]
    return seg_to_verts, num_verts

In [10]:
a, b = read_segmentation(file)

In [2]:
scannetv2_label = '/home/duccd/Desktop/InstanceRefer/data/scannet/meta_data/scannetv2-labels.combined.tsv'

In [3]:
lines = [line.rstrip() for line in open(scannetv2_label)]
lines = lines[1:]
nyu40ids2class = {}

In [ ]:
    def _get_nyu40id2class(self):
        lines = [line.rstrip() for line in open(os.path.join(CONF.PATH.SCANNET, 'meta_data/scannetv2-labels.combined.tsv'))]
        lines = lines[1:]
        nyu40ids2class = {}
        for i in range(len(lines)):
            label_classes_set = set(self.type2class.keys())
            elements = lines[i].split('\t')
            nyu40_id = int(elements[4])
            nyu40_name = elements[7]
            if nyu40_id in self.nyu40ids:
                if nyu40_name not in label_classes_set:
                    nyu40ids2class[nyu40_id] = self.type2class["others"]
                else:
                    nyu40ids2class[nyu40_id] = self.type2class[nyu40_name]

        return nyu40ids2class